In [ ]:
import numpy as np
from qc_lbm_linear import QcLbmD2Q9, BoundaryType
from qiskit_aer import StatevectorSimulator
from qiskit.quantum_info import Statevector
from qiskit import transpile, ClassicalRegister
from qiskit.circuit import QuantumCircuit

nx = 8
ny = 8
n_step = 500
u_0 = 0.02
tau = 0.9
omega = 1 / tau

nq = 9
cx = [0, 1, 0, -1, 0, 1, -1, -1, 1]
cy = [0, 0, 1, 0, -1, 1, 1, -1, -1]
wi = [4/9, 1/9, 1/9, 1/9, 1/9, 1/36, 1/36, 1/36, 1/36]

feq = np.zeros((nx, ny, nq))
for iq in range(nq):
    feq[:,:,iq]=wi[iq]

# quantum LBM simulation
normalization_factors = []
qc_lbm = QcLbmD2Q9()
num_f_coords = qc_lbm.num_max_variables * qc_lbm.nx * qc_lbm.ny
f_ini = qc_lbm.FlattenInitialField(feq)
qc_collision_matrix = qc_lbm.ComputeCollisionMatrix(omega)
qc_lbm.SefBounceBackBoundaryForCollisionMatrix(BoundaryType.y_max, nq, u_0, qc_collision_matrix, f_ini)
qc_lbm.SefBounceBackBoundaryForCollisionMatrix(BoundaryType.y_min, nq+1, 0, qc_collision_matrix, f_ini)


simulator = StatevectorSimulator()
f_real = f_ini
for t in range(n_step):
    [flatten_scaler, qc] = qc_lbm.InitialEncoding(f_real)
    matrix_scaler = qc_lbm.MatrixMultiplier(qc_collision_matrix, qc)
    qc_lbm.Stream(qc)
    
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit).result()

    statevector = np.array(result.get_statevector(compiled_circuit))
    f_real = np.real(statevector[:qc_lbm.num_max_variables*qc_lbm.nx*qc_lbm.ny])*flatten_scaler*matrix_scaler


In [ ]:
# classical LBM simulation
f=feq.copy()
fprop=feq.copy()
for t in range(n_step):
    rho = np.sum(fprop, axis=2)

    u = np.sum(fprop[:, :, [1, 5, 8]], axis=2) - np.sum(fprop[:, :, [3, 6, 7]], axis=2)
    v = np.sum(fprop[:, :, [2, 5, 6]], axis=2) - np.sum(fprop[:, :, [4, 7, 8]], axis=2)
    
    for k in range(nq):
        feq[:, :, k] = wi[k] * (rho + 3 * (u * cx[k] + v * cy[k]))

    f = (1 - omega) * fprop + omega * feq
    
    fprop_new = np.zeros_like(fprop)
    for k in range(nq):
        for j in range(ny):
            for i in range(nx):
                newx = (i + cx[k]) % nx
                newy = (j + cy[k]) % ny
                fprop_new[newx, newy, k] = f[i, j, k]
    fprop = fprop_new

    fprop[:, ny-1, 4] = f[:, ny -1, 2]
    fprop[:, ny-1, 7] = f[:, ny -1, 5] - (1/6) * u_0
    fprop[:, ny-1, 8] = f[:, ny -1, 6] + (1/6) * u_0

    fprop[:, 0, 2] = f[:, 0, 4]
    fprop[:, 0, 5] = f[:, 0, 7]
    fprop[:, 0, 6] = f[:, 0, 8]

num_var = nq + 7
f_flattened = np.zeros((num_var*nx*ny))
for iy in range(ny):
    for ix in range(nx):
        node_index = iy * nx * num_var + ix * num_var
        f_flattened[node_index:node_index + nq] = fprop[ix, iy, :]
for ix in range(nx):
    node_index = (ny - 1) * nx * num_var + ix * num_var
    f_flattened[node_index + nq] = u_0

f_size = nx*ny*num_var
i_count = 0
f_qc = np.zeros(f_size)
sv = Statevector(statevector)
num_qubits = sv.num_qubits
for i, amp in enumerate(sv.data):
    basis_state = format(i, f'0{num_qubits}b')
    if i_count < f_size:
        f_qc[i_count] = amp.real*flatten_scaler*matrix_scaler
    i_count += 1


In [ ]:
# plot
u_analy = np.zeros(ny)
for j in range(ny):
    u_analy[j]= u_0/ny*(j + 0.5)

f_reshape =np.reshape(f_real, (ny, nx, num_var))
u_reshape = np.sum(f_reshape[:, :, [1, 5, 8]], axis=2) - np.sum(f_reshape[:, :, [3, 6, 7]], axis=2)

error = np.zeros(nx)
for i in range(nx):
    error[i] = np.sqrt(np.sum((u_reshape[:, i] - u_analy) ** 2)) / np.sqrt(np.sum(u_analy ** 2))

L2 = np.mean(error)

print(f"error: {L2}")

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 14
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
plt.figure(figsize=(10, 6))
plt.plot(range(ny), u_reshape[:, 3] - u_analy, marker='o', linestyle='-.', color='blue')
plt.xlabel(r'Index ($i_y$)')
plt.ylabel('Error')
plt.grid(True)
plt.tight_layout()
plt.show()

